In [176]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2
import scipy.signal
import scipy.ndimage
import sys
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")
import helpers
import glm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
# TODO: UNUSED CODE, IGNORE THIS CELL
# # Just to visualize images
# fig, ax = plt.subplots()
# ax.axis("off")
# ax.set_title("Projected points")
# ax.imshow(windowed(testing_0, 8).reshape((128,128)), cmap="gray")
# # ax.imshow(testing_0, cmap="gray")
# 
# fig.tight_layout()
# plt.show()
# print(testing_0)

#     # return img.reshape((int(img.shape[0] / window_size), int(img.shape[1] / window_size), window_size, window_size))
#     
# def windowed(img, window_size):
#     return ressample(img, window_size)
# def ressample(arr, N):
#     A = []
#     for v in np.vsplit(arr, arr.shape[0] // N):
#         A.extend([*np.hsplit(v, arr.shape[0] // N)])
#     return np.array(A)

In [173]:
# WINDOWING FUNCTIONS
# assuming for now that window size fits perfectly into img, and img is 2d np array perfect square  
def windowed(img, window_size):
    return np.lib.stride_tricks.as_strided(img, shape=(int(img.shape[0] / window_size), int(img.shape[1] / window_size), window_size, window_size), strides =(img.strides[0] * window_size, img.strides[1] * window_size, img.strides[0], img.strides[1]) , writeable=False)


In [177]:
def moving_window_array(array: torch.Tensor, window_size, overlap) -> torch.Tensor:
    shape = array.shape
    strides = (
        shape[-1] * (window_size - overlap),
        (window_size - overlap),
        shape[-1],
        1
    )
    shape = (
        int((shape[-2] - window_size) / (window_size - overlap)) + 1,
        int((shape[-1] - window_size) / (window_size - overlap)) + 1,
        window_size,
        window_size,
    )
    return torch.as_strided(
        array, size=shape, stride=strides 
    ).reshape(-1, window_size, window_size)

In [194]:
testtesttorch = torch.from_numpy(testtest)
print(testtesttorch)
print(moving_window_array(testtesttorch, 4, 3))

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62, 63]], dtype=torch.int32)
tensor([[[ 0,  1,  2,  3],
         [ 8,  9, 10, 11],
         [16, 17, 18, 19],
         [24, 25, 26, 27]],

        [[ 1,  2,  3,  4],
         [ 9, 10, 11, 12],
         [17, 18, 19, 20],
         [25, 26, 27, 28]],

        [[ 2,  3,  4,  5],
         [10, 11, 12, 13],
         [18, 19, 20, 21],
         [26, 27, 28, 29]],

        [[ 3,  4,  5,  6],
         [11, 12, 13, 14],
         [19, 20, 21, 22],
         [27, 28, 29, 30]],

        [[ 4,  5,  6,  7],
         [12, 13, 14, 15],
         [20, 21, 22, 23],
         [28, 29, 30, 31]],

        [[ 8,  9, 10, 11],
         [16, 17, 18, 19],
         [24, 25, 26, 27],
         [32,

In [174]:
testing_0 = helpers.imread_normalized_float_grayscale(os.path.join("test", "0.jpg"))
testing_0 = np.where(testing_0 > 0.5, 1.0, 0.0)  # TEMPORARY, TO MAKE TEST MORE IDEAL
testing_1 = helpers.imread_normalized_float_grayscale(os.path.join("test", "1.jpg"))
testing_1 = np.where(testing_1 > 0.5, 1.0, 0.0)  # TEMPORARY, TO MAKE TEST MORE IDEAL
testtest = np.array(np.split(np.arange(64), 8))
def calculate(img1, img2, window_size, overlap):
    # Assumptions for simple version: ignoring restrictions and bug checks, assuming window size fits into img with no remainder, overlap is half of window size, images are numpy arrays and perfect squares
    # split img1 into windows:
    img1_windowed = windowed(img1, window_size)
    img2_windowed = windowed(img2, window_size)
    return img1_windowed
print(calculate(testing_0, testing_1, 16, 1))



[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]
(8, 8)
(32, 4)
[[[[ 0  1  2  3]
   [ 8  9 10 11]
   [16 17 18 19]
   [24 25 26 27]]

  [[ 4  5  6  7]
   [12 13 14 15]
   [20 21 22 23]
   [28 29 30 31]]]


 [[[32 33 34 35]
   [40 41 42 43]
   [48 49 50 51]
   [56 57 58 59]]

  [[36 37 38 39]
   [44 45 46 47]
   [52 53 54 55]
   [60 61 62 63]]]]
(2, 2, 4, 4)
(128, 16, 32, 4)
